In [135]:
import requests
import pandas as pd
import numpy as np
import json
from dotenv import load_dotenv
import os
import time

In [113]:
api_key = {}
try:
    with open('api_key.json') as file:
        api_key = json.load(file)
except FileNotFoundError:
    print("Error: api_key.json file not found.")

In [117]:
# The team_id_grab() function grabs the teams id,name,conference and conference position and stores it in a DataFrame
    
def team_id_dict():
    url = "https://v1.basketball.api-sports.io/standings?league=12&season=2022-2023"

    payload = {}
    headers = {
        'x-rapidapi-key': api_key['x-rapidapi-key'],
        'x-rapidapi-host': 'v1.basketball.api-sports.io'
    }
     # request.request sends a request of a specified method to the url (in this case the method is GET)
    response = requests.request("GET", url, headers=headers, data=payload)

    # json.loads deserialises a JSON object to a standard python object (good for parsing)
    # also it turns nulls to a None python object
    standings = json.loads(response.text)

    # team_lvl (team level) grabs the section of the returned data that we need to parse through to collect 
    # info on the team
    team_lvl = standings['response'][0]
       
    # Creating a DataFrame to store the team API ids
    team_ids_df = pd.DataFrame(columns = ['Team', 'Team ID', 'Conference', 'Conference Position'])
     
    # Creating variables that hold the requests daily limit remaining (100 per day) 
    # and the per minute requests remaining (10 per minute)
    daily_calls_remaining = response.headers['x-ratelimit-requests-remaining']
    perMinute_calls_remaining = response.headers['X-RateLimit-Remaining']
    
    print(f'Daily calls remaining = {daily_calls_remaining}')
    print(f'Calls left per minute = {perMinute_calls_remaining}\n')

    team_id_dict = {}
    
    for team in team_lvl:
        if team['group']['name'] == "Western Conference" or team['group']['name'] == "Eastern Conference":
            t_id = int(team['team']['id'])
            t_name = team['team']['name']
            team_id_dict[t_id] = t_name
        else:
            break
            
    return team_id_dict


def team_id_list():
    url = "https://v1.basketball.api-sports.io/standings?league=12&season=2022-2023"

    payload = {}
    headers = {
        'x-rapidapi-key': api_key['x-rapidapi-key'],
        'x-rapidapi-host': 'v1.basketball.api-sports.io'
    }
    # request.request sends a request of a specified method to the url (in this case the method is GET)
    response = requests.request("GET", url, headers=headers, data=payload)

    # json.loads deserialises a JSON object to a standard python object (good for parsing)
    # also it turns nulls to a None python object
    standings = json.loads(response.text)

    # team_lvl (team level) grabs the section of the returned data that we need to parse through to collect 
    # info on the team
    team_lvl = standings['response'][0]
       
    # Creating a DataFrame to store the team API ids
    team_ids_df = pd.DataFrame(columns = ['Team', 'Team ID', 'Conference', 'Conference Position'])
     
    # Creating variables that hold the requests daily limit remaining (100 per day) 
    # and the per minute requests remaining (10 per minute)
    daily_calls_remaining = response.headers['x-ratelimit-requests-remaining']
    perMinute_calls_remaining = response.headers['X-RateLimit-Remaining']
    
    print(f'Daily calls remaining = {daily_calls_remaining}')
    print(f'Calls left per minute = {perMinute_calls_remaining}\n')
    
    team_id_list = []
    
    for team in team_lvl:
        if team['group']['name'] == "Western Conference" or team['group']['name'] == "Eastern Conference":
            id = int(team['team']['id'])

            team_id_list.append(id)
        else:
            break
    
    return team_id_list
    
def team_position_df():
    # request.request sends a request of a specified method to the url (in this case the method is GET)
    url = "https://v1.basketball.api-sports.io/standings?league=12&season=2022-2023"

    payload = {}
    headers = {
        'x-rapidapi-key': api_key['x-rapidapi-key'],
        'x-rapidapi-host': 'v1.basketball.api-sports.io'
    }
    response = requests.request("GET", url, headers=headers, data=payload)

    # json.loads deserialises a JSON object to a standard python object (good for parsing)
    # also it turns nulls to a None python object
    standings = json.loads(response.text)

    # team_lvl (team level) grabs the section of the returned data that we need to parse through to collect 
    # info on the team
    team_lvl = standings['response'][0]
       
    # Creating a DataFrame to store the team API ids
    team_position_df = pd.DataFrame(columns = ['Team','Team ID', 'Conference', 'Conference Position'])
     
    # Creating variables that hold the requests daily limit remaining (100 per day) 
    # and the per minute requests remaining (10 per minute)
    daily_calls_remaining = response.headers['x-ratelimit-requests-remaining']
    perMinute_calls_remaining = response.headers['X-RateLimit-Remaining']
    
    print(f'Daily calls remaining = {daily_calls_remaining}')
    print(f'Calls left per minute = {perMinute_calls_remaining}\n')

    df_row = 0
    
    for team in team_lvl:
        if team['group']['name'] == "Western Conference" or team['group']['name'] == "Eastern Conference":
            t_id = int(team['team']['id'])
            t_name = team['team']['name']
            t_conference = team['group']['name']
            t_position = str(team['position'])

            team_position_df.loc[df_row] = [t_name, t_id, t_conference, t_position]

            df_row += 1
        else:
            break

    team_position_df.set_index('Team', inplace = True)

    return team_position_df


def team_stats_df_all_season(team_id_list):

    games_played_df = pd.DataFrame(columns = ['Team', 'Home games', 'Away games', 'Total games'])
    games_won_df = pd.DataFrame(columns = ['Team', 'Home wins', 'Away wins', 'Total wins'])
    games_lost_df = pd.DataFrame(columns = ['Team', 'Home loses', 'Away loses', 'Total loses'])
    points_for_df = pd.DataFrame(columns = ['Team', 'Home points for', 'Away points for', 'Total points for'])
    points_against_df = pd.DataFrame(columns = ['Team', 'Home points against', 'Away points against', 'Total points against'])
    
    df_row = 0    
    
    for id in team_id_list:
        t_id = str(id)
        # API URL for NBA team data via api-basketball.com
        url = "https://v1.basketball.api-sports.io/statistics?season=2022-2023&team="+t_id+"&league=12"

        payload = {}
        headers = {
            'x-rapidapi-key': api_key['x-rapidapi-key'],
            'x-rapidapi-host': 'v1.basketball.api-sports.io'
        }

        daily_calls_remaining = 1
        perMinute_calls_remaining = 1

        # if statement to make sure I don't go over the API's free version daily and per minunte limit
        if perMinute_calls_remaining > 0 and daily_calls_remaining > 0: 
            response = requests.request("GET", url, headers=headers, data=payload)
            # request.request sends a request of a specified method to the url (in this case the method is GET)

            # Parses the JSON
            team_statistics = json.loads(response.text)
            # Use the print statment below to check the layout of the data pulled with the GET request
            # print(json.dumps(team_statistics, indent=4))

            daily_calls_remaining = int(response.headers['x-ratelimit-requests-remaining'])
            perMinute_calls_remaining = int(response.headers['X-RateLimit-Remaining'])

            # extracts team name from data
            team_name = team_statistics["response"]["team"]["name"]
            
            # extracts data for GAMES PLAYED DF
            games_home = team_statistics["response"]["games"]["played"]["home"]
            games_away = team_statistics["response"]["games"]["played"]["away"]
            games_all = team_statistics["response"]["games"]["played"]["all"]

            # extracts data for GAMES LOST DF
            won_home = team_statistics["response"]["games"]["wins"]["home"]["total"]
            won_away = team_statistics["response"]["games"]["wins"]["away"]["total"]
            total_won = team_statistics["response"]["games"]["wins"]["all"]["total"]

            # extracts data for GAMES LOST DF
            lost_home = team_statistics["response"]["games"]["loses"]["home"]["total"]
            lost_away = team_statistics["response"]["games"]["loses"]["away"]["total"]
            total_lost = team_statistics["response"]["games"]["loses"]["all"]["total"]

            # extracts data for POINTS FOR DF
            points_for_home = team_statistics["response"]["points"]["for"]["total"]["home"]
            points_for_away = team_statistics["response"]["points"]["for"]["total"]["away"]
            points_for_total = team_statistics["response"]["points"]["for"]["total"]["all"]

            # extracts data for POINTS FOR DF
            points_against_home = team_statistics["response"]["points"]["against"]["total"]["home"]
            points_against_away = team_statistics["response"]["points"]["against"]["total"]["away"]
            points_against_total = team_statistics["response"]["points"]["against"]["total"]["all"]

            games_played_df.loc[df_row] = [team_name, games_home, games_away, games_all]
            games_won_df.loc[df_row] = [team_name, won_home, won_away, total_won]
            games_lost_df.loc[df_row] = [team_name, lost_home, lost_away, total_lost]
            points_for_df.loc[df_row] = [team_name, points_for_home, points_for_away, points_for_total]
            points_against_df.loc[df_row] = [team_name, points_against_home, points_against_away, points_against_total]
              
            df_row += 1

            print(games_played_df)
            print(games_won_df)
            print(games_lost_df)
            print(points_for_df)
            print(points_against_df)
            
            print('=====================================================================================')
            
            print('Daily calls remaining = ' + str(daily_calls_remaining))
            print('Calls left per minute = ' + str(perMinute_calls_remaining), '\n')

            time.sleep(7)
            # Sleep for 7 seconds to avoid having more than 10 calls per minute as I am using the free API  
        else:
            break

    return games_played_df, games_won_df, games_lost_df, points_for_df, points_against_df



In [129]:
def games_played_df_rectifier(t_id, team_position_df, game):
    team_name = team_position_df.index[team_position_df['Team ID'] == int(t_id)].tolist()[0]
    home_condition = game["teams"]["home"]['name']
    away_condition = game["teams"]["away"]['name']
    if team_name == home_condition:
        games_played_df["Home games"] = games_played_df["Home games"].apply(lambda x: x-1 if team_name == games_played_df["Team"] else x=x)
        games_played_df["Total games"] = games_played_df["Total games"].apply(lambda x: x-1 if team_name == games_played_df["Team"] else x=x)
        display(games_played_df)
    elif team_name == away_condition:
        games_played_df["Away games"] = games_played_df["Away games"].apply(lambda x: x-1 if team_name == games_played_df["Team"] else x=x
        games_played_df["Total games"] = games_played_df["Total games"].apply(lambda x: x-1 if team_name == games_played_df["Team"] else x=x)
        display(games_played_df)
        
    
def regular_season_all_rectifier(team_id_list, games_played_df, games_won_df, games_lost_df, points_for_df, points_against_df):

    for id in team_id_list:
        t_id = str(id)
        # API URL for NBA team data via api-basketball.com
        url = "https://v1.basketball.api-sports.io/games?league=12&season=2022-2023&team="+t_id+"&timezone=America/New_York"
        

        payload = {}
        headers = {
            'x-rapidapi-key': api_key['x-rapidapi-key'],
            'x-rapidapi-host': 'v1.basketball.api-sports.io'
        }

        daily_calls_remaining = 1
        perMinute_calls_remaining = 1

        # if statement to make sure I don't go over the API's free version daily and per minunte limit
        if perMinute_calls_remaining > 0 and daily_calls_remaining > 0: 
            response = requests.request("GET", url, headers=headers, data=payload)
            # request.request sends a request of a specified method to the url (in this case the method is GET)

            # Parses the JSON
            games = json.loads(response.text)
            # Use the print statment below to check the layout of the data pulled with the GET request
            # print(json.dumps(team_statistics, indent=4))

            # game_lvl (game level) grabs the section of the returned data that we need to parse through to collect 
            # info on the games played by a team
            game_lvl = games['response']

            for game in game_lvl:
                games_played_df_rectifier(t_id, team_position_df, game)
            

            daily_calls_remaining = int(response.headers['x-ratelimit-requests-remaining'])
            perMinute_calls_remaining = int(response.headers['X-RateLimit-Remaining'])

            
            print('=====================================================================================')
            
            print('Daily calls remaining = ' + str(daily_calls_remaining))
            print('Calls left per minute = ' + str(perMinute_calls_remaining), '\n')

            time.sleep(7)
            # Sleep for 7 seconds to avoid having more than 10 calls per minute as I am using the free API  
        else:
            break

    return games_played_df_reg, games_won_df_reg, games_lost_df_reg, points_for_df_reg, points_against_df

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (4267833442.py, line 6)

In [119]:
team_id_list = team_id_list()
team_position_df = team_position_df()

Daily calls remaining = 62
Calls left per minute = 9

Daily calls remaining = 61
Calls left per minute = 8



In [130]:
games_played_df, games_won_df, games_lost_df, points_for_df, points_against_df = team_stats_df_all_season(team_id_list)

             Team  Home games  Away games  Total games
0  Denver Nuggets          54          53          107
             Team  Home wins  Away wins  Total wins
0  Denver Nuggets         45         27          72
             Team  Home loses  Away loses  Total loses
0  Denver Nuggets           9          26           35
             Team  Home points for  Away points for  Total points for
0  Denver Nuggets             6332             5998             12330
             Team  Home points against  Away points against  \
0  Denver Nuggets                 5831                 6069   

   Total points against  
0                 11900  
Daily calls remaining = 36
Calls left per minute = 9 

                Team  Home games  Away games  Total games
0     Denver Nuggets          54          53          107
1  Memphis Grizzlies          46          47           93
                Team  Home wins  Away wins  Total wins
0     Denver Nuggets         45         27          72
1  Memphis Grizzli

Denver Nuggets
<class 'str'>


In [131]:
display(games_played_df)

,Team,Home games,Away games,Total games
0,Denver Nuggets,54,53,107
1,Memphis Grizzlies,46,47,93
2,Sacramento Kings,47,46,93
3,Phoenix Suns,48,48,96
4,Golden State Warriors,50,50,100
5,Los Angeles Clippers,46,44,90
6,New Orleans Pelicans,43,45,88
7,Los Angeles Lakers,54,51,105
8,Minnesota Timberwolves,45,49,94
9,Oklahoma City Thunder,42,46,88


In [132]:
games_played_df_reg, games_won_df_reg, games_lost_df_reg, points_for_df_reg, points_against_df_reg = regular_season_all_rectifier(team_id_list, games_played_df, games_won_df, games_lost_df, points_for_df, points_against_df)

0     53
1     45
2     46
3     47
4     49
5     45
6     42
7     53
8     44
9     41
10    41
11    42
12    41
13    42
14    43
15    46
16    53
17    47
18    45
19    48
20    44
21    55
22    45
23    44
24    43
25    42
26    42
27    42
28    43
29    42
Name: Home games, dtype: int64
0     52
1     46
2     45
3     47
4     49
5     43
6     44
7     50
8     48
9     45
10    42
11    42
12    43
13    42
14    42
15    44
16    51
17    48
18    44
19    47
20    44
21    55
22    46
23    42
24    43
25    42
26    43
27    42
28    42
29    42
Name: Away games, dtype: int64
0     52
1     44
2     45
3     46
4     48
5     44
6     41
7     52
8     43
9     40
10    40
11    41
12    40
13    41
14    42
15    45
16    52
17    46
18    44
19    47
20    43
21    54
22    44
23    43
24    42
25    41
26    41
27    41
28    42
29    41
Name: Home games, dtype: int64
0     51
1     45
2     44
3     46
4     48
5     42
6     43
7     49
8     47
9     44
10    4

KeyboardInterrupt: 

In [133]:
display(games_played_df_reg)

NameError: name 'games_played_df_reg' is not defined

In [10]:
print(team_id_list)
print(team_position_df)

Daily calls remaining = 93
Calls left per minute = 9

[139, 146, 157, 155, 141, 144, 150, 145, 149, 152, 138, 160, 156, 142, 158, 148, 133, 154, 137, 151, 134, 147, 132, 159, 136, 161, 153, 143, 135, 140]
Daily calls remaining = 92
Calls left per minute = 8

                      Team  Team ID          Conference Conference Position
0           Denver Nuggets      139  Western Conference                   1
1        Memphis Grizzlies      146  Western Conference                   2
2         Sacramento Kings      157  Western Conference                   3
3             Phoenix Suns      155  Western Conference                   4
4    Golden State Warriors      141  Western Conference                   5
5     Los Angeles Clippers      144  Western Conference                   6
6     New Orleans Pelicans      150  Western Conference                   7
7       Los Angeles Lakers      145  Western Conference                   8
8   Minnesota Timberwolves      149  Western Conference  

KeyboardInterrupt: 